# Simple Linear versus Ridge Regression 

## Requirement 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [0]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold
import pandas as pd
from sklearn.metrics import mean_squared_error
import collections

In [0]:
from sklearn.model_selection import cross_val_score

###  Importing the dataset

In [0]:
# Import the boston dataset from sklearn
# Load dataset to some variable 
# boston_data = .....
boston_data = load_boston()
boston_data.keys()



dict_keys(['data', 'target', 'feature_names', 'DESCR', 'filename'])

Exploration of the Data Set

In [0]:
print("Shape of the data set: ")
print(boston_data.data.shape)
print("Actual Data Set: ")
print(boston_data.data)
# boston_data.data


Shape of the data set: 
(506, 13)
Actual Data Set: 
[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]


In [0]:
print("Features of the data set to run regression on: ")
boston_data.feature_names

Features of the data set to run regression on: 


array(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
       'TAX', 'PTRATIO', 'B', 'LSTAT'], dtype='<U7')

In [0]:
boston_data.DESCR

".. _boston_dataset:\n\nBoston house prices dataset\n---------------------------\n\n**Data Set Characteristics:**  \n\n    :Number of Instances: 506 \n\n    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.\n\n    :Attribute Information (in order):\n        - CRIM     per capita crime rate by town\n        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.\n        - INDUS    proportion of non-retail business acres per town\n        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)\n        - NOX      nitric oxides concentration (parts per 10 million)\n        - RM       average number of rooms per dwelling\n        - AGE      proportion of owner-occupied units built prior to 1940\n        - DIS      weighted distances to five Boston employment centres\n        - RAD      index of accessibility to radial highways\n        - TAX      full-value property-tax rate per $10,000

In [0]:
boston_pd = pd.DataFrame(boston_data.data)
print(boston_pd.head())

        0     1     2    3      4   ...   8      9     10      11    12
0  0.00632  18.0  2.31  0.0  0.538  ...  1.0  296.0  15.3  396.90  4.98
1  0.02731   0.0  7.07  0.0  0.469  ...  2.0  242.0  17.8  396.90  9.14
2  0.02729   0.0  7.07  0.0  0.469  ...  2.0  242.0  17.8  392.83  4.03
3  0.03237   0.0  2.18  0.0  0.458  ...  3.0  222.0  18.7  394.63  2.94
4  0.06905   0.0  2.18  0.0  0.458  ...  3.0  222.0  18.7  396.90  5.33

[5 rows x 13 columns]


###Requirement 2: Features
Below is a brief explanation of the different characteristics of the data set. We have 13 features, such as crime per capita (CRIM) and INDUS (proportion of non-retail business acres per town) , that will be used to determine house prices in Boston. Our goal for both the ridge regression model and the linear regression model is to find the best w for each feature in order to predict house prices. Such models will be trained with a dataset of 506 data points, and we will use the closed form solution for both models to train our model and determine our w values.

In [0]:
#  Create X and Y variables - X holding the .data and Y holding .target 
X = boston_data.data
y = boston_data.target
#  Reshape Y to be a rank 2 matrix using y.reshape()
y = y.reshape(X.shape[0],1)
# Observe the number of features and the number of labels
print('The number of features is: ', X.shape[1])
# Printing out the features
print('The features: ', boston_data.feature_names)
# The number of examples
print('The number of examples in ourdataset: ', X.shape[0])
# Observing the first 2 rows of the data
print("First two rows of data: ")
print(X[0:2])


The number of features is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
The number of examples in ourdataset:  506
First two rows of data: 
[[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


# Your code goes here

In [0]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values



def get_coeff_ridge_normaleq(X_train, y_train, alpha):
    # use np.linalg.pinv(...)
    # W = ((X_T . X + alpha * I )^-1) . X_T . y
    bias = np.ones((X_train.shape[0],1))
    X_train = np.c_[X_train,bias]
    X_T = X_train.transpose()
    X_T_dot_X = X_T.dot(X_train)

    alpha_times_identity = alpha * np.identity (X_T_dot_X.shape[1])
    temp_1 = np.linalg.pinv(X_T_dot_X + alpha_times_identity)
    temp_2 = temp_1.dot(X_T)
    w = temp_2.dot(y_train)


    return w

In [0]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values

def get_coeff_linear_normaleq(X_train, y_train):
    bias = np.ones((X_train.shape[0],1))
    X_train= np.c_[X_train,bias]
    w = np.linalg.inv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
    return w

In [0]:
# Define the evaluate_err_ridge function.
# Return the train_error and test_error values


def evaluate_err(X_train, X_test, y_train, y_test, w): 
#     pred_train=prediction using w and X_tran+np.mean(y_train)
#     pred_test=prediction using w and X_test
#     remember to add the mean back

    bias = np.ones((X_train.shape[0],1))
    X_train = np.c_[X_train,bias]
    pred_train = X_train.dot(w)
    testbias = np.ones((X_test.shape[0],1))
    X_test = np.c_[X_test,testbias]  
    pred_test = X_test.dot(w) 
    train_error = np.square(np.subtract(y_train,pred_train)).mean()
    test_error = np.square(np.subtract(y_test,pred_test)).mean()
    
    return train_error, test_error

In [0]:
# Finish writting the k_fold_cross_validation function. 
# Returns the average training error and average test error from the k-fold cross validation
# Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, X, y, alpha,model_type):
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = 0
    total_E_val_train = 0
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        # Subtract y_train_mean from y_train and y_test
        y_train_mean = np.mean(y_train)
        y_train = y_train - y_train_mean
        y_test = y_test - y_train_mean
        
        # Scaling the data matrix
        # Using scaler=preprocessing.StandardScaler().fit(...)
        # And scaler.transform(...)
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        # Determine the training error and the test error
        # Use get_coeff_linear_normaleq or get_coeff_ridge_normaleq to get w
        # And use evaluate_err()
        if model_type == "linear":
          w_linear = get_coeff_linear_normaleq(X_train, y_train)
          temp1,temp2 = evaluate_err(X_train,X_test, y_train, y_test ,w_linear)
          total_E_val_test += temp2
          total_E_val_train +=temp1

        else:
          w_ridge = get_coeff_ridge_normaleq( X_train, y_train , alpha)
          temp1,temp2= evaluate_err(X_train,X_test,y_train,y_test,w_ridge)
          total_E_val_test += temp2
          total_E_val_train += temp1
       ##############

    #compute average for all Ks
    total_E_val_test = total_E_val_test/k
    total_E_val_train = total_E_val_train/k
    return  total_E_val_test, total_E_val_train
    


###Requirement 3
In the cells below,  both the linear regression model and ridge regression model is implemented using their respective closed form solutions. In order to evaluate the performance of the models,a k-cross validation will be performed, a statistical method that divides the subset into K subparts, training the model with K-1 and testing it with K. 10 proved to be a really good number for k to partition the data set by.  After several rounds of random data partitioning and validation, the k-cross validation reports an an error of 21% during training and 23% during testing for the linear regression model

In [0]:
# print the error for the both linear regression and ridge regression
# the error should include both training error and testing error


#Linear
total_E_val_test_linear, total_E_val_train_linear = k_fold_cross_validation(10,X,y,0,"linear")
print("Average Linear Test Error: ",total_E_val_test_linear)
print("Average Linear Train Error: ",total_E_val_train_linear)





Average Linear Test Error:  23.63606860542814
Average Linear Train Error:  21.806183575851062


###Requirement 4 and 5

The same procedure will be done for the ridge regression model and validating such using the k-crosss validation procedure. In addition, different lambda values, the coefficient that applies the penalty for the ridge model, will be tested in order to determine the most performant one. Below are the errors for the different lambda, and the one with the least error turned out to be alpha = 10, with a Test Error of 23.68% and  Train Error:21.89

In [0]:
# use the model to predict the new test case.

alpha_values = np.logspace(1, 7, num=13)
for i in alpha_values:
  total_E_val_test_ridge, total_E_val_train_ridge = k_fold_cross_validation(10,X,y,i,"ridge")
  print("---------------------"+str(i)+"---------------------------------")
  print("Test Error: "+str(total_E_val_test_ridge)+" Train Error: "+str(total_E_val_train_ridge))



---------------------10.0---------------------------------
Test Error: 23.68858300163874 Train Error: 21.892901157570186
---------------------31.622776601683793---------------------------------
Test Error: 24.01784029738615 Train Error: 22.285444055697496
---------------------100.0---------------------------------
Test Error: 25.29385255369514 Train Error: 23.725488384882077
---------------------316.22776601683796---------------------------------
Test Error: 29.457296222896787 Train Error: 28.16655409854012
---------------------1000.0---------------------------------
Test Error: 39.48949335939869 Train Error: 38.53214872734357
---------------------3162.2776601683795---------------------------------
Test Error: 54.64719107794949 Train Error: 54.00702649955449
---------------------10000.0---------------------------------
Test Error: 69.71851750884511 Train Error: 69.2597817132532
---------------------31622.776601683792---------------------------------
Test Error: 78.92162110435103 Train 

###Requirement 6: Polynomial Features
In order to improve the results presented above, both the ridge regression and linear regression models will be evaluated using the Polynomial Features of D = 2. This will hopefully predict housing prices better as it will better model any non-linear relationships presented by some features in the data set. 

In [0]:
# Create a PolynomialFeatures object with degree = 2. Using PolynomialFeatures(degree=2)
# Transform X and save it into X_2 using poly.fit_transform(X)
# Simply copy Y into Y_2 

poly =PolynomialFeatures(degree = 2)
X_2 = poly.fit_transform(X)
y_2 = y
print(X)
print(X_2)

[[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]
[[1.00000000e+00 6.32000000e-03 1.80000000e+01 ... 1.57529610e+05
  1.97656200e+03 2.48004000e+01]
 [1.00000000e+00 2.73100000e-02 0.00000000e+00 ... 1.57529610e+05
  3.62766600e+03 8.35396000e+01]
 [1.00000000e+00 2.72900000e-02 0.00000000e+00 ... 1.54315409e+05
  1.58310490e+03 1.62409000e+01]
 ...
 [1.00000000e+00 6.07600000e-02 0.00000000e+00 ... 1.57529610e+05
  2.23851600e+03 3.18096000e+01]
 [1.00000000e+00 1.09590000e-01 0.00000000e+00 ... 1.54802902e+05
  2.54955600e+03 4.19904000e+01]
 [1.00000000e+00 4.74100000e-02 0.00000000e+00 ... 1.575

In [0]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
print(X_2.shape)
print(y_2.shape)

(506, 105)
(506, 1)


In [0]:
print("Ridge Regression Error Using Polynomial Features:")
for i in alpha_values:
  total_E_val_test_ridge, total_E_val_train_ridge = k_fold_cross_validation(10,X_2,y_2,i,"ridge")
  print("---------------------"+str(i)+"---------------------------------")
  print("Test Error: "+str(total_E_val_test_ridge)+" Train Error: "+str(total_E_val_train_ridge))

Ridge Regression Error Using Polynomial Features:
---------------------10.0---------------------------------
Test Error: 13.476138001136254 Train Error: 10.049055874118876
---------------------31.622776601683793---------------------------------
Test Error: 15.8296019690513 Train Error: 12.751706269046773
---------------------100.0---------------------------------
Test Error: 18.980018815009952 Train Error: 16.2226905932108
---------------------316.22776601683796---------------------------------
Test Error: 22.068692308062726 Train Error: 19.700253646409855
---------------------1000.0---------------------------------
Test Error: 26.21847559440485 Train Error: 24.28745798310889
---------------------3162.2776601683795---------------------------------
Test Error: 34.580266018800174 Train Error: 33.08666831985053
---------------------10000.0---------------------------------
Test Error: 47.78952058756749 Train Error: 46.761999359414084
---------------------31622.776601683792-----------------

In [0]:
#Linear
print("Linear Error Using Polynomial Features")
total_E_val_test_linear, total_E_val_train_linear = k_fold_cross_validation(10,X_2,y_2,0,"linear_regression")
print("Average Linear Test Error: ",total_E_val_test_linear)
print("Average Linear Train Error: ",total_E_val_train_linear)

Linear Error Using Polynomial Features
Average Linear Test Error:  11.854968234646984
Average Linear Train Error:  5.808820816012461


### Requirement 7: Which model performed best?

After predicting the data using both the standard linear regression  model and the ridge regression model, the linear regression model performed best for both the standard X and the polynomial X_2. Creating the  polynomial features of degree 2 resulted in a more performant linear regression model  with an 5.8% error during testing and 11.85% error during training. This was surprising, as the data was both centered and normalized before performing the k-cross validation step which might indicate that the model was finetuned enough. However, the polynomial features revealed that there were some non linear relationships between the features that needeed to be modeled with a polynomial curve instead of a straight line.


